<a href="https://colab.research.google.com/github/neel26desai/transformers_and_finetuning_with_LLM/blob/main/Textbooks_are_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

In this notebook, we be looking at the "TextBook is all you need case study". The main idea is we can have smaller LLMs learn some capabilities on larger LLMs, by fine tuning them on good quality textbooks on that capability and generating newwer textbooks through larger LLMS on that topic and use them for finetuning the smaller LLMs.


In this notebook, we plan on teaching smaller LLM like gpt2, ability to answer questions about terraform using a text book generated by Chat GPT4o

# Synthetic text book Generations

In [1]:
!pip install openai langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19


In [8]:
from langchain_openai import ChatOpenAI

In [3]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


### Just checking the Connection

In [9]:
llm = ChatOpenAI(model_name="gpt-4o-mini",api_key=userdata.get('open_ai') )

In [10]:
res = llm.invoke("Hello")

In [11]:
res.content

'Hello! How can I assist you today?'

In [12]:
#this structure is generated through a previous conversation I had with GPT
book_structure = {
    "Introduction": [
        "What is Terraform?",
        "History and Evolution of Terraform",
        "Why Learn Terraform Today?",
        "How Terraform Fits in the DevOps Ecosystem"
    ],
    "Terraform Fundamentals": [
        "Core Concepts: Providers, Resources, Modules, State",
        "Step-by-Step Guide to Install Terraform",
        "First Terraform Configuration",
        "Understanding HCL (HashiCorp Configuration Language)",
        "Terraform CLI Basics: Commands and Workflow"
    ],
    "Intermediate Topics": [
        "State Management: Local vs Remote State",
        "Terraform Modules: Creating and Using Reusable Components",
        "Variables and Outputs: Best Practices",
        "Debugging and Troubleshooting Terraform"
    ],
    "Advanced Topics": [
        "State Management and Remote State Backends",
        "Terraform Workspaces for Environment Management",
        "Dynamic Blocks and Loops in Terraform",
        "Custom Providers and Plugins",
        "Terraform Enterprise and Cloud"
    ],
    "Terraform on AWS": [
        "Setting Up AWS Credentials for Terraform",
        "Using AWS Provider: Core Concepts",
        "Deploying EC2 Instances with Terraform",
        "Managing S3 Buckets and IAM Roles",
        "Building a VPC with Terraform",
        "Scaling with Auto Scaling Groups and Load Balancers",
        "Integrating Terraform with AWS Services like RDS and Lambda"
    ],
    "Terraform on Azure": [
        "Setting Up Azure Credentials for Terraform",
        "Using AzureRM Provider: Core Concepts",
        "Deploying Virtual Machines with Terraform",
        "Managing Azure Storage and Networking",
        "Building a Secure Azure Infrastructure with NSGs and VNets",
        "Scaling with Azure Kubernetes Service (AKS)",
        "Integrating Terraform with Azure Services like CosmosDB and Azure Functions"
    ],
    "Real-World Use Cases": [
        "Infrastructure as Code (IaC) in Action",
        "Multi-Cloud Deployments with Terraform",
        "Managing Kubernetes Clusters with Terraform",
        "Integrating Terraform with CI/CD Pipelines",
        "Building Scalable and Secure Infrastructure"
    ],
    "Best Practices and Tips": [
        "Structuring Your Terraform Codebase",
        "Version Control for Terraform State and Configurations",
        "Testing Infrastructure Code with Terraform",
        "Managing Terraform in Large Teams",
        "Performance Optimization Tips"
    ],
    "Conclusion": [
        "Future of Terraform and IaC",
        "Learning Resources and Next Steps",
        "Final Thoughts"
    ]
}


In [13]:
def generate_chapter(chapter_title, topics):
    content = f"# {chapter_title}\n\n"
    for topic in topics:
        prompt = f"""
        Write a detailed section for a book titled "Mastering Terraform".
        Chapter: {chapter_title}
        Topic: {topic}

        Provide an in-depth explanation with examples, step-by-step guidance, and any relevant code snippets. Ensure the content is beginner-friendly yet insightful.
        """
        response = llm.invoke(prompt)
        content += f"## {topic}\n\n"
        content += response.content + "\n\n"
    return content

In [15]:
def save_book(book_structure, file_name="terraform_book.txt"):
    with open(file_name, "w") as file:
        for chapter_title, topics in book_structure.items():
            chapter_content = generate_chapter(chapter_title, topics)
            file.write(chapter_content)
    print(f"Book saved to {file_name}")


In [16]:
# Generate and save the book
save_book(book_structure)


Book saved to terraform_book.txt


In [17]:
with open("terraform_book.txt", "r") as file:
    book_content = file.read()
print(f"Character length of the book: {len(book_content)}")

Character length of the book: 276453


# Dependecides for model

In [2]:
# Step 1: Install Dependencies
!pip install datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Step 2: Import Libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

In [4]:
import torch

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Base Case

In [7]:
prompt = "What are some providers for Azure in terraform "
input_ids = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, max_new_tokens=124)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What are some providers for Azure in terraform ?"

"I'm not sure what they're doing, but I'm sure they're doing some things that are very useful for us. We're looking at a lot of different things, but we're not sure what they're doing."

"I'm not sure what they're doing, but I'm sure they're doing some things that are very useful for us. We're looking at a lot of different things, but we're not sure what they're doing."

"I'm not sure what they're doing, but I'm sure they're doing some things that are very useful for us


## Preparing the data for finetuning

In [10]:
with open("terraform_book.txt", "r") as file:
    book_content = file.read()

In [25]:
import re
split_content = re.split(r"(?<=[.!?])\s+", book_content)  # Split on sentence boundaries

In [26]:
# Load into Hugging Face's Dataset format
dataset = Dataset.from_dict({"text": split_content})
dataset = dataset.train_test_split(test_size=0.2)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=50,return_special_tokens_mask=True)

In [29]:
tokenized_datasets = dataset.map(tokenize_function, batched=True,remove_columns=["text"])
tokenized_datasets.set_format(type='torch', device=device)


Map:   0%|          | 0/1542 [00:00<?, ? examples/s]

Map:   0%|          | 0/386 [00:00<?, ? examples/s]

In [33]:
# Step 6: Define Training Arguments
training_args = TrainingArguments(
        output_dir="./gpt2-trained",
        num_train_epochs=2,
        per_device_train_batch_size=64, # can do batch size 32 or 64
        gradient_accumulation_steps=4,  # Accumulate gradients
        learning_rate=2e-5,  # Lower learning rate
        warmup_ratio=0.1,    # Add warmup
        weight_decay=0.01,   # Add weight decay
        logging_steps=1,
        save_steps=50,
        no_cuda=False if torch.cuda.is_available() else True,
        report_to="wandb",
        logging_dir="./logs",
        logging_first_step=True,
        logging_nan_inf_filter=True,
        warmup_steps=100,
    )

# Step 7: Fine-Tuning
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     tokenizer=tokenizer,
# )

In [34]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    TrainerCallback
)
from datasets import Dataset, load_dataset
import wandb


In [35]:
# Custom callback for better progress tracking
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero:
            if logs is not None:
                wandb.log(logs)


In [36]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

In [37]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[LoggingCallback()]
    )

<ipython-input-37-d0300afeb1f2>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [42]:
trainer.train()

Step,Training Loss
1,4.195800
2,4.311600
3,4.176100
4,4.224900
5,4.145300
6,4.254100
7,4.218500
8,4.182100
9,4.274900
10,4.146900


TrainOutput(global_step=12, training_loss=4.197723786036174, metrics={'train_runtime': 42.7911, 'train_samples_per_second': 72.071, 'train_steps_per_second': 0.28, 'total_flos': 76907635200000.0, 'train_loss': 4.197723786036174, 'epoch': 1.92})

In [43]:
prompt = '''The following are terraform providers for Azure'''
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=50, num_return_sequences=1)
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("Generated Answer:\n")
print(generated_code)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Answer:

The following are terraform providers for Azure.

The terraform provider is a service that provides a terraform for a given resource.
The terraform provider is a service that provides a terraform for a given resource


As we have only trained for 2 epochs, we do see a change from base case but not major one. Let us try for a few more epochs

In [47]:
# Step 6: Define Training Arguments
training_args = TrainingArguments(
        output_dir="./gpt2-trained",
        num_train_epochs=10,
        per_device_train_batch_size=32, # can do batch size 32 or 64
        gradient_accumulation_steps=4,  # Accumulate gradients
        learning_rate=2e-5,  # Lower learning rate
        warmup_ratio=0.1,    # Add warmup
        weight_decay=0.01,   # Add weight decay
        logging_steps=1,
        save_steps=50,
        no_cuda=False if torch.cuda.is_available() else True,
        report_to="wandb",
        logging_dir="./logs",
        logging_first_step=True,
        logging_nan_inf_filter=True,
        warmup_steps=100,
    )


In [48]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[LoggingCallback()]
    )

<ipython-input-48-d0300afeb1f2>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [49]:
trainer.train()

Step,Training Loss
1,3.791600
2,3.882100
3,3.939500
4,3.975000
5,3.802800
6,3.793400
7,3.819400
8,3.856400
9,3.781900
10,3.728600


TrainOutput(global_step=120, training_loss=3.018138412634532, metrics={'train_runtime': 231.6956, 'train_samples_per_second': 66.553, 'train_steps_per_second': 0.518, 'total_flos': 385967116800000.0, 'train_loss': 3.018138412634532, 'epoch': 9.795918367346939})

In [50]:
prompt = '''The following are terraform providers for Azure'''
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=50, num_return_sequences=1)
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("Generated Answer:\n")
print(generated_code)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Answer:

The following are terraform providers for Azure CLI:

Azure CLI:

Azure CLI is a powerful tool for managing infrastructure and services. It allows you to manage infrastructure and services in a declarative, declarative manner
